<a href="https://colab.research.google.com/github/vrmusketeers/FYP-2021/blob/master/notebooks/neural_net/VG_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!rm -rf '/content/drive/Shareddrives/Final Project/FmriTrainTestData/VG16-Data/Output/train/.ipynb_checkpoints'
!rm -rf '/content/drive/Shareddrives/Final Project/FmriTrainTestData/VG16-Data/Output/test/.ipynb_checkpoints'
!rm -rf '/content/drive/Shareddrives/Final Project/FmriTrainTestData/VG16-Data/Output/val/.ipynb_checkpoints'

In [ ]:
import os 
import zipfile 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
import matplotlib.pyplot as plt

In [ ]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [ ]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory('/content/drive/Shareddrives/Final Project/FmriTrainTestData/VG16-Data/Output/train', batch_size = 20, class_mode = 'binary', target_size = (224, 224))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory('/content/drive/Shareddrives/Final Project/FmriTrainTestData/VG16-Data/Output/val',  batch_size = 20, class_mode = 'binary', target_size = (224, 224))

Found 6968 images belonging to 2 classes.
Found 871 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (224, 224, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
vgghist = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 30, epochs = 10)

Epoch 1/10
30/30 [==============================] - 692s 23s/step - loss: 1.0735 - acc: 0.4800 - val_loss: 0.7217 - val_acc: 0.5350
Epoch 2/10
30/30 [==============================] - 257s 9s/step - loss: 0.7737 - acc: 0.4900 - val_loss: 0.6941 - val_acc: 0.5029
Epoch 3/10
30/30 [==============================] - 234s 8s/step - loss: 0.7375 - acc: 0.5300 - val_loss: 0.6983 - val_acc: 0.5373
Epoch 4/10
30/30 [==============================] - 211s 7s/step - loss: 0.7120 - acc: 0.5117 - val_loss: 0.6885 - val_acc: 0.5396
Epoch 5/10
30/30 [==============================] - 196s 6s/step - loss: 0.6951 - acc: 0.5650 - val_loss: 0.6996 - val_acc: 0.4834
Epoch 6/10
30/30 [==============================] - 167s 6s/step - loss: 0.7055 - acc: 0.5383 - val_loss: 0.7023 - val_acc: 0.5373
Epoch 7/10
30/30 [==============================] - 166s 5s/step - loss: 0.6974 - acc: 0.5317 - val_loss: 0.6969 - val_acc: 0.5373
Epoch 8/10
30/30 [==============================] - 138s 5s/step - loss: 0.7068 - 